In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Read the data
train_data = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")


In [3]:
train_data

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

def preprocess_data(data):
    le = LabelEncoder()
    ss = StandardScaler()
    
    # Encode categorical features
    data['HomePlanet'] = le.fit_transform(data['HomePlanet'])
    data['Cabin'] = le.fit_transform(data['Cabin'])
    data['Destination'] = le.fit_transform(data['Destination'])
    data['Name'] = le.fit_transform(data['Name'])
    
    # Scale numerical features
    numerical_features = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    data[numerical_features] = ss.fit_transform(data[numerical_features])
    
    # Handle missing values
    data = data.fillna(data.median())  # You can also use `data.median()` or a constant value like `data.fillna(0)`
    
    return data

train_data = preprocess_data(train_data)
test_data = preprocess_data(test_data)



In [5]:
# Split the training data into training and validation sets
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Prepare the features and target variable
X_train = train_data.drop(['PassengerId', 'Transported'], axis=1)
y_train = train_data['Transported']
X_val = val_data.drop(['PassengerId', 'Transported'], axis=1)
y_val = val_data['Transported']
X_test = test_data.drop('PassengerId', axis=1)


In [6]:
# Train the RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Make predictions on the validation set
y_val_pred = clf.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_val_pred)
confusion = confusion_matrix(y_val, y_val_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:", confusion)


Accuracy: 0.7843588269120184
Confusion Matrix: [[674 187]
 [188 690]]


In [7]:
# 1. Calculate additional metrics
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_val, y_val_pred)
recall = recall_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)




Precision: 0.7867730900798175
Recall: 0.785876993166287
F1-score: 0.7863247863247864


In [8]:
# 2. Feature importance
importances = clf.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances})
print(feature_importance_df.sort_values('Importance', ascending=False))



         Feature  Importance
2          Cabin    0.167562
11          Name    0.126805
1      CryoSleep    0.114438
4            Age    0.107794
9            Spa    0.102871
10        VRDeck    0.087783
6    RoomService    0.086082
7      FoodCourt    0.080044
8   ShoppingMall    0.068682
0     HomePlanet    0.033428
3    Destination    0.022710
5            VIP    0.001802


In [9]:
# 3. Hyperparameter tuning
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
print("Best Parameters:", best_params)



Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}


In [10]:
# 4. Train the model on the entire training set
X_full_train = pd.concat([X_train, X_val])
y_full_train = pd.concat([y_train, y_val])

clf = RandomForestClassifier(**best_params, random_state=42)
clf.fit(X_full_train, y_full_train)




RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)

In [11]:
# 5. Make predictions on the test set
y_test_pred = clf.predict(X_test)


In [12]:
submission = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Transported': y_test_pred})
submission.to_csv('submission.csv', index=False)
print('Submission succesfully saved')
submission.head()

Submission succesfully saved


,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
